In [1]:


import matplotlib.pyplot as plt
from scipy.io import wavfile
from pydub import AudioSegment
import numpy as np
import struct

import torch
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
# from model import Net


In [2]:
v1 = AudioSegment.from_file('audio/v1.m4a')
v2 = AudioSegment.from_file('audio/v2.m4a')
t1 = AudioSegment.from_file('audio/t1.m4a')
t2 = AudioSegment.from_file('audio/t2.m4a')


In [3]:

def draw(file, answer):
    frame_rate = file.frame_rate
    raw_data = file.get_array_of_samples()
    # plt.figure(figsize=(20, 3))
    # plt.plot(r)
    # plt.show()
    result = []
    t = (frame_rate / 1000 * 250)
    for i in range(int(len(raw_data) / t) - 1):
        r = file.get_array_of_samples()[int(t * i):int(t * (i + 1))]
        freq_sepctrum = np.fft.fft(r)
        freq = np.fft.fftfreq(len(freq_sepctrum), d= 1 / frame_rate)
        len_data = int(len(freq) / 2)
        x = freq[:len_data][:32*32]
        y = np.abs(freq_sepctrum[:len_data][:32*32])
        # plt.figure(figsize=(10,3)); plt.plot(x, y); plt.show()
        y = (y - y.min()) / (y.max() - y.min())
        result.append({"data": torch.from_numpy(np.array(y).astype('float32')), "answer": torch.from_numpy(np.array(answer))})
        
    return np.array(result)
    

In [4]:

vv1 = draw(v1, 0)
vv2 = draw(v2, 1)
tt1 = draw(t1, 0)
tt2 = draw(t2, 1)

vv3 = np.append(vv1, vv2)
tt3 = np.append(tt1, tt2)



In [83]:

device = torch.device("cuda")

train_kwargs = {'batch_size': 2}
test_kwargs = {'batch_size': 2}
cuda_kwargs = {'num_workers': 1,
                'pin_memory': True,
                'shuffle': True}

train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, padding='same')
        self.conv2 = nn.Conv2d(32, 64, 3, 1, padding='same')
        self.dropout = nn.Dropout2d(0.25)
        # (입력 뉴런, 출력 뉴런)
        self.fc1 = nn.Linear(3136, 1000)    # 7 * 7 * 64 = 3136
        self.fc2 = nn.Linear(1000, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [125]:

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for idx, elem in enumerate(train_loader):
        data, target = elem["data"], elem["answer"]
        data = data[:784].reshape((1,1,28,28))
        target = target.reshape((1))
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, idx * len(data), len(train_loader),
                100. * idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, elem in enumerate(test_loader):
            data, target = elem["data"], elem["answer"]
            data = data[:784].reshape((1,1,28,28))
            target = target.reshape((1))
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader),
        100. * correct / len(test_loader)))


In [126]:
# for i in vv3:
#     plt.title(i['answer'])
#     plt.imshow(i['data'].reshape((32,32)))
#     plt.show()


In [128]:

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(100):
    train(model, device, vv3, optimizer, epoch)
    test(model, device, tt3)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")


Train Epoch: 0 [0/90 (0%)]	Loss: 2.320688
Train Epoch: 0 [10/90 (11%)]	Loss: 0.000172
Train Epoch: 0 [20/90 (22%)]	Loss: 0.000086
Train Epoch: 0 [30/90 (33%)]	Loss: 0.001192
Train Epoch: 0 [40/90 (44%)]	Loss: 0.000015
Train Epoch: 0 [50/90 (56%)]	Loss: 0.006520
Train Epoch: 0 [60/90 (67%)]	Loss: 0.002049
Train Epoch: 0 [70/90 (78%)]	Loss: 0.000930
Train Epoch: 0 [80/90 (89%)]	Loss: 0.000181

Test set: Average loss: 4.2828, Accuracy: 44/83 (53%)

Train Epoch: 1 [0/90 (0%)]	Loss: 8.517733
Train Epoch: 1 [10/90 (11%)]	Loss: 0.007742
Train Epoch: 1 [20/90 (22%)]	Loss: 0.007609
Train Epoch: 1 [30/90 (33%)]	Loss: 0.030961
Train Epoch: 1 [40/90 (44%)]	Loss: 0.001054
Train Epoch: 1 [50/90 (56%)]	Loss: 0.018617
Train Epoch: 1 [60/90 (67%)]	Loss: 0.003400
Train Epoch: 1 [70/90 (78%)]	Loss: 0.008706
Train Epoch: 1 [80/90 (89%)]	Loss: 0.000161

Test set: Average loss: 3.6353, Accuracy: 44/83 (53%)

Train Epoch: 2 [0/90 (0%)]	Loss: 6.832904
Train Epoch: 2 [10/90 (11%)]	Loss: 0.048273
Train Epoch: 2

In [100]:
from torchvision import datasets, transforms

train_data = datasets.MNIST(root = '../data',
                            train=True,
                            download=True)
                            
# train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                        #    batch_size = 64, shuffle = True)
# first_batch = train_loader.__iter__().__next__()


In [105]:
train_data[0][0].size

(28, 28)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/collate.py", line 142, in <listcomp>
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibili